<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/encoder_decoderV2_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import pandas as pd
import numpy as np
import sys
import string
from string import digits
from pandas import Panel
from sklearn.datasets import make_circles
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os
import re
import io
import time
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import xml.etree.ElementTree as et 
import tensorflow as tf 
import tensorflow_addons as tfa
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import GRU, Bidirectional, LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import pickle as pkl

from keras.layers import Dense
from keras.models import Sequential, Input, Model
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

from keras import backend as K 
K.clear_session() 


In [5]:
# import pandas as pd 
import pandas as pd 
   
# Takes the file's folder 
#filepath = r"C:Documents\dataset.csv"

filepath = r"/content/drive/My Drive/Colab Notebooks/complex/updated complex/dataset.csv" 
# read the CSV file 

#lines = pd.read_csv(filepath, encoding='ascii')
lines = pd.read_csv(filepath, sep='delimiter', header=None, engine='python') 


In [6]:
print(len(lines)) 
lines.columns = ['qa']
list(lines.columns)

4040


['qa']

In [7]:
lines[['question','answer']] = lines.qa.apply(lambda x: pd.Series(str(x).split("?")))

In [8]:
list(lines.columns)

['qa', 'question', 'answer']

In [9]:
for col in lines.columns: 
    if 'qa' in col: 
        del lines[col] 
  

In [10]:
lines.iloc[:3]

,question,answer
0,What is (are) abdominal wall defect,",An opening in the abdomen through which vario..."
1,What are the treatments for abdominal wall defect,",Diagnostic Tests-Drug Therapy-Surgery and Reh..."
2,What is (are) Osteoporosis,",A Bone Disease that thins and weakens the bon..."


In [11]:
lines.columns = ['source', 'target']

In [12]:
list(lines.columns)

['source', 'target']

In [13]:
lines['source'].iloc[2000]

'What are the treatments for late-infantile neuronal ceroid lipofuscinosis '

In [14]:
lines['target'].iloc[2000]

',"An eye disorder that affects the clear, outer covering of the eye called the cornea. "'

In [15]:
lines['target'] = lines['target'].fillna('').apply(str)

In [16]:
# convert source and target text to Lowercase 
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

# Remove quotes from source and target text
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))

#s = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", " ", x)

lines.source=lines.source.apply(lambda x: re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]',r' ',x))
lines.target=lines.target.apply(lambda x: re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]',r' ',x))

'''
# create a set of all special characters
special_characters= set(string.punctuation)

# Remove all the special characters
lines.source=lines.source.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]',r'',x))
lines.target=lines.target.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]',r'',x))
'''
# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))


In [17]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,source,target
2194,what is the risk for salivary gland cancer,START_ being exposed to certain types of radia...
3626,what are the treatments for vitiligo,START_ diagnostic tests drug therapy surgery a...
1500,what is are intestinal pseudo obstruction,START_ condition characterized by impairment o...
747,how many people are affected by piebaldism,START_ the prevalence of piebaldism is unknown...
3912,what are the genetic changes related to q dele...,START_ q deletion syndrome is caused by a dele...
2056,what is parathyroid cancer,START_ a rare disease in which cancer cells fo...


In [18]:
lines['source'].iloc[2086]

'how to diagnose colon cancer'

In [19]:
lines['target'].iloc[2086]

'START_ digital rectal exam biopsy _END'

In [20]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

len(source_words), len(target_words)

(1270, 3635)

In [21]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)
target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 19
 Max length of the target sentence 31


In [22]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [23]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

{1: 'a', 2: 'aarskog', 3: 'ab', 4: 'abdominal', 5: 'abetalipoproteinemia', 6: 'abnormal', 7: 'absent', 8: 'abuse', 9: 'acatalasemia', 10: 'accumulation', 11: 'acetylglutamate', 12: 'achondrogenesis', 13: 'achondroplasia', 14: 'achromatopsia', 15: 'acid', 16: 'acidemia', 17: 'acidosis', 18: 'aciduria', 19: 'acral', 20: 'acrofacial', 21: 'acromelic', 22: 'acromicric', 23: 'actin', 24: 'activated', 25: 'acute', 26: 'acyl', 27: 'adams', 28: 'adcy', 29: 'adenine', 30: 'adenosine', 31: 'adenylosuccinate', 32: 'adermatoglyphia', 33: 'adhesion', 34: 'adiposis', 35: 'adolescent', 36: 'adrenal', 37: 'adrenoleukodystrophy', 38: 'adult', 39: 'adults', 40: 'affected', 41: 'african', 42: 'agammaglobulinemia', 43: 'age', 44: 'aggravated', 45: 'aggregate', 46: 'aicardi', 47: 'aids', 48: 'alagille', 49: 'albinism', 50: 'albright', 51: 'aldrich', 52: 'alexander', 53: 'alg', 54: 'alkaptonuria', 55: 'allan', 56: 'allergic', 57: 'alopecia', 58: 'alpers', 59: 'alpha', 60: 'amelia', 61: 'american', 62: 'amyl

In [24]:
#Shuffle the data
lines = shuffle(lines)

In [25]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train.shape, X_test.shape

((2828,), (1212,))

In [26]:
# https://stackoverflow.com/questions/54880279/how-to-handle-invalid-argument-error-in-keras
# Input tokens for encoder
num_encoder_tokens=len(source_words) +1
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [27]:
def generate_batch(X = X_train, y = y_train, batch_size = 25):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                        encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

Attention model

In [28]:
# -*- coding: utf-8 -*-
"""attention.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1XrjPL3O_szhahYZW0z9yhCl9qvIcJJYW
"""

import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


In [29]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 25
epochs = 100
latent_dim= 500

Encoder Decoder Model

In [30]:

# Encoder 
encoder_inputs = Input(shape=(max_source_length,)) 
enc_emb = Embedding(num_encoder_tokens, latent_dim,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(num_decoder_tokens, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 


In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 19)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 19, 500)      635500      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 19, 500), (N 2002000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
_______________________________________________________________________________________

In [32]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, epochs=epochs)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
113/113 [==============================] - 24s 215ms/step - loss: 1.8210 - acc: 0.0959 - precision: 0.2135 - recall: 0.0906 - val_loss: 1.5213 - val_acc: 0.1476 - val_precision: 0.2778 - val_recall: 0.2253
Epoch 2/100
113/113 [==============================] - 22s 197ms/step - loss: 1.2794 - acc: 0.1695 - precision: 0.2981 - recall: 0.3128 - val_loss: 1.3087 - val_acc: 0.1802 - val_precision: 0.5525 - val_recall: 0.3437
Epoch 3/100
113/113 [==============================] - 22s 197ms/step - loss: 1.0977 - acc: 0.1894 - precision: 0.3421 - recall: 0.3773 - val_loss: 1.2500 - val_acc: 0.1899 - val_precision: 0.2600 - val_recall: 0.3689
Epoch 4/100
113/113 [==============================] - 22s 197ms/step - loss: 0.9764 - acc: 0.2016 - precision: 0.4766 - recall: 0.4162 - val_loss: 1.2226 - val_acc: 0.1923 - val_precision: 0.3537 - val_recall: 0.3984
Epoch 5/100
113/113 [==============================]

In [33]:
model_json = model.to_json()
with open("NMT_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("NMT_model_weight.h5")
print("Saved model to disk")

Saved model to disk


In [34]:
# loading the model architecture and asigning the weights
json_file = open('NMT_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json, custom_objects={'AttentionLayer': AttentionLayer})
# load weights into new model
model_loaded.load_weights("NMT_model_weight.h5")

In [35]:
# encoder inference
encoder_inputs = model_loaded.input[0]  #loading encoder_inputs
encoder_outputs, state_h, state_c = model_loaded.layers[6].output #loading encoder_outputs

print(encoder_outputs.shape)

encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(32,latent_dim))

# Get the embeddings of the decoder sequence
decoder_inputs = model_loaded.layers[3].output

print(decoder_inputs.shape)
dec_emb_layer = model_loaded.layers[5]

dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_lstm = model_loaded.layers[7]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_layer = model_loaded.layers[8]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])

concate = model_loaded.layers[9]
decoder_inf_concat = concate([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_dense = model_loaded.layers[10]
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

(None, 19, 500)
(None, None)


In [36]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']

# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
        
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 110):
            stop_condition = True

# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

# Update states
        e_h, e_c = h, c
    return decoded_sentence

In [37]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
i=-1

In [38]:
#Make predictions on the test dataset
i+= 1
for k in range(i,i+40):
    #print("\n",k)
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input Source Question:', X_test[k:k+1].values[0],'?')
    print('Actual Target Answer:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Target Answer:', decoded_sentence[:-4])
    print("\n")

Input Source Question: what are the treatments for weaver syndrome ?
Actual Target Answer:  diagnostic tests drug therapy surgery and rehabilitation genetic counseling 
Predicted Target Answer:  diagnostic tests drug therapy surgery and rehabilitation genetic counseling 


Input Source Question: what are the genetic changes related to phosphoribosylpyrophosphate synthetase superactivity ?
Actual Target Answer:  certain mutations in the prps gene cause prs superactivity 
Predicted Target Answer:  mutations in the manb gene cause alpha mannosidosis 


Input Source Question: what are the treatments for cerebellar hypoplasia ?
Actual Target Answer:  there is no standard course of treatment for cerebellar hypoplasia 
Predicted Target Answer:  there is no standard course of treatment for sotos syndrome 


Input Source Question: what is the outlook for krabbe disease ?
Actual Target Answer:  krabbe disease in infants is generally fatal before age 
Predicted Target Answer:  the prognosis for t

In [39]:
#Make predictions on the train dataset

train_gen = generate_batch(X_train, y_train, batch_size = 1)
x=-1

In [40]:
x+=1
for j in range(x,x+40):
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input Source Question:', X_train[j:j+1].values[0],'?')
    print('Actual Target Answer:', y_train[j:j+1].values[0][6:-4])
    print('Predicted Target Answer:', decoded_sentence[:-4])
    print("\n")

Input Source Question: how to diagnose vulvar cancer ?
Actual Target Answer:  chest x ray biopsy computed tomography scan 
Predicted Target Answer:  chest x ray biopsy computed tomography scan 


Input Source Question: what is the risk for salivary gland cancer ?
Actual Target Answer:  exposed to certain types of radiation 
Predicted Target Answer:  exposed to certain types of radiation 


Input Source Question: what are the treatments for problems with taste ?
Actual Target Answer:  relief is possible although there is no treatment relief from taste disorders is possible 
Predicted Target Answer:  relief is possible although there is no treatment relief from taste disorders is possible 


Input Source Question: what are the treatments for l syndrome ?
Actual Target Answer:  diagnostic tests drug therapy surgery and rehabilitation genetic counseling palliative care 
Predicted Target Answer:  diagnostic tests drug therapy surgery and rehabilitation genetic counseling palliative care 


